In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from string import punctuation

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import re
import numpy as np

In [2]:
!pip install gensim

In [5]:
df = pd.read_csv('D:/Papers.csv')
df.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [6]:
def prepare_data(df):
    data=df['paper_text']
    useless_words=list(stopwords.words('english')+list(punctuation))
    
    lemmatizer=WordNetLemmatizer()
    corpus=data.tolist()
    final_corpus=[]
    
    for i in range(len(corpus)):
        word=word_tokenize(corpus[i].lower())
        word=[
            lemmatizer.lemmatize(y) for y in word if y not in useless_words
        ]
        j=" ".join(word)
        j=re.sub(r'[^\w\s]', " ", j)
        j=re.sub("(^|\W)\d+"," ",j)
        
        final_corpus.append(j)
    
    return final_corpus

In [9]:
final_corpus=prepare_data(df)

id_=df['id']
train_df={
    'id':id_,
    'content':final_corpus,
}

train_df=pd.DataFrame(train_df)
train_df.head()

,id,content
0,1,self organization associative database appli...
1,10,mean field theory layer iv visual cortex app...
2,100,storing covariance associative long term pot...
3,1000,bayesian query construction neural network mod...
4,1001,neural network ensemble cross validation activ...


In [10]:
final_data=train_df['content'].map(word_tokenize)

In [11]:
dictionary = gensim.corpora.Dictionary(final_data)

In [12]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 _l
1 a
2 abolish
3 abstract
4 accelerate
5 acceptable
6 accomplishes
7 accordingly
8 achieve
9 achieved
10 actual


In [13]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [14]:
bow_corpus = [dictionary.doc2bow(doc) for doc in final_data]

In [15]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [17]:
for index, score in sorted(lda_model_tfidf[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.474081814289093	 
Topic: 0.004*"neuron" + 0.003*"spike" + 0.002*"synaptic" + 0.001*"kernel" + 0.001*"cell" + 0.001*"layer" + 0.001*"stimulus" + 0.001*"image" + 0.001*"firing" + 0.001*"node"

Score: 0.4462987184524536	 
Topic: 0.002*"neuron" + 0.002*"image" + 0.002*"spike" + 0.001*"kernel" + 0.001*"tree" + 0.001*"layer" + 0.001*"classifier" + 0.001*"label" + 0.001*"stimulus" + 0.001*"clustering"

Score: 0.07909981906414032	 
Topic: 0.003*"policy" + 0.002*"image" + 0.002*"action" + 0.001*"game" + 0.001*"neuron" + 0.001*"agent" + 0.001*"kernel" + 0.001*"layer" + 0.001*"player" + 0.001*"node"
